In [1]:
pip install datasets

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
     ---------------------------------------- 0.0/71.4 kB ? eta -:--:--
     ----------------- ---------------------- 30.7/71.4 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 71.4/71.4 kB 784.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/485.4 kB ? eta -:--:--
   ------ --------------------------------- 81.9/485.4 kB 4.5 MB/s eta 0:00:01
   ------------- -------------------------- 163.8/485.4 kB 2.0 MB/s eta 0:00:01
   --------------------- ------------------ 266.2/485.4 kB 2.7 MB/s eta 0:00:01
   ---------------------------------------  481.3/485.4 kB 2.7 MB/s eta 0:00:01
   --------------------------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset

news = load_dataset('multi_news', split='test', trust_remote_code=True)

In [3]:
news.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


In [4]:
train_news = news.train_test_split(test_size = .2)

In [ ]:
pip install transformers accelerate

In [7]:
from transformers import AutoTokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained('t5-small')    # t5-small is pretrained on hugging face

In [9]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized = train_news.map(preprocess_function , batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForSeq2Seq , AutoModelForSeq2SeqLM , Seq2SeqTrainingArguments, Seq2SeqTrainer

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer , model = 't5-small')

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

In [15]:
training_args = Seq2SeqTrainingArguments(
output_dir="./results",
evaluation_strategy="epoch",
learning_rate=2e-5,
per_device_train_batch_size=10,
per_device_eval_batch_size=10,
weight_decay=0.01,
save_total_limit=3,
num_train_epochs=10,
fp16=True,
)

C:\Users\Utkarsh yadav\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized['train'],
    eval_dataset = tokenized['test'],
    tokenizer = tokenizer,
    data_collator = data_collator
)

C:\Users\Utkarsh yadav\AppData\Local\Temp\ipykernel_15928\4183548326.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: utkarshyadav72a (utkarshyadav72a-jaypee-university-of-engineering-and-tec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


In [ ]:
def pred(document):
  device = model.device
  tokenized= tokenizer([document], return_tensors = 'pt')
  print(tokenized)
  tokenized ={k:v.to(device) for k , v in tokenized.items()}
  results = model.generate(**tokenized, max_length = 128)
  results = results.to('cpu')
  pred = tokenizer.decode(results[0])
  return pred


In [ ]:
document = "National Archives Yes, it's that time again, folks. It's the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you're here, why don't you sign up to follow us on Twitter. Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth. The unemployment rate dipped, but mostly because more Americans stopped looking for work. The Labor Department says the economy added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate fell to 8.2 percent, the lowest since January 2009. The rate dropped because fewer people searched for jobs. The official unemployment tally only includes those seeking work. The economy has added 858,000 jobs since December _ the best four months of hiring in two years. But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending."
human_summary = """– The unemployment rate dropped to 8.2% last month, but the economy only added 120,000 jobs, when 203,000 new jobs had been predicted, according to today's jobs report. Reaction on the Wall Street Journal's MarketBeat Blog was swift: "Woah!!! Bad number." The unemployment rate, however, is better news; it had been expected to hold steady at 8.3%. But the AP notes that the dip is mostly due to more Americans giving up on seeking employment."""

In [ ]:
summary = pred(document)

In [ ]:
summary

In [ ]:

# Save the trained model
model.save('./text_summarizer.h5')
print("Model saved successfully as 'text_summarizer.h5'")
